### Clone Repository and Install Dependencies

In [ ]:
!git clone https://github.com/pratyushghosh/cs4243_miniproject_team16

In [ ]:
!pip install tensorflow==1.15.5 pandas==1.2.0 opencv-python matplotlib tensorflow-object-detection-api

In [ ]:
cd /content/cs4243_miniproject_team16

In [ ]:
mkdir test_images

### Build OpenPose

In [ ]:
!git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose

In [ ]:
cd openpose

In [ ]:
!git submodule update --init --recursive --remote

In [ ]:
!mkdir build

In [ ]:
cd build

In [ ]:
!sudo apt-get install protobuf-compiler libprotobuf-dev libgoogle-glog-dev

In [ ]:
!cmake -DBUILD_PYTHON=TRUE -S /content/cs4243_miniproject_team16/openpose/ -B /content/cs4243_miniproject_team16/openpose/build/

In [ ]:
!make -j`nproc`

In [ ]:
!cp /content/cs4243_miniproject_team16/openpose/build/examples/openpose/openpose.bin /content/cs4243_miniproject_team16/openpose/openpose.bin

### Place Input Images Inside /content/cs4243_miniproject_team16/test_images

### Run Model

In [ ]:
cd /content/cs4243_miniproject_team16/

In [ ]:
!python model_a_code.py

In [ ]:
cd /content/cs4243_miniproject_team16/openpose

In [ ]:
!./openpose.bin --image_dir /content/cs4243_miniproject_team16/openpose_input --write_json /content/cs4243_miniproject_team16/model_b_input/ --display 0 --render_pose 0

In [ ]:
cd /content/cs4243_miniproject_team16/

In [ ]:
!python model_b_code.py

### Normal Outputs Are In /content/cs4243_miniproject_team16/normal

### Threat Outputs Are In /content/cs4243_miniproject_team16/threat

### Carrying Outputs Are In /content/cs4243_miniproject_team16/carrying

### Generate Confusion Matrix (Only For Filenames Containing Label)

In [ ]:
import numpy as np
import glob
import os
from sklearn import metrics

y_true = []
y_predicted = []

for file in glob.glob('/content/cs4243_miniproject_team16/normal/*.jpg'):
 if('normal' in os.path.basename(file)):
   y_true.append('normal')
   y_predicted.append('normal')
 elif('carrying' in os.path.basename(file)):
   y_true.append('carrying')
   y_predicted.append('normal')
 elif('threat' in os.path.basename(file)):
   y_true.append('threat')
   y_predicted.append('normal')

for file in glob.glob('/content/cs4243_miniproject_team16/carrying/*.jpg'):
 if('normal' in os.path.basename(file)):
   y_true.append('normal')
   y_predicted.append('carrying')
 elif('carrying' in os.path.basename(file)):
   y_true.append('carrying')
   y_predicted.append('carrying')
 elif('threat' in os.path.basename(file)):
   y_true.append('threat')
   y_predicted.append('carrying')

for file in glob.glob('/content/cs4243_miniproject_team16/threat/*.jpg'):
 if('normal' in os.path.basename(file)):
   y_true.append('normal')
   y_predicted.append('threat')
 elif('carrying' in os.path.basename(file)):
   y_true.append('carrying')
   y_predicted.append('threat')
 elif('threat' in os.path.basename(file)):
   y_true.append('threat')
   y_predicted.append('threat')

confusion_matrix = metrics.confusion_matrix(y_true, y_predicted, labels = ['normal', 'carrying', 'threat'])
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['normal', 'carrying', 'threat'])
cm_display.plot()

correct = 0;
for i in range(len(y_true)):
  if (y_true[i] == y_predicted[i]):
    correct = correct + 1

accuracy = correct/len(y_true)
print("Accuracy: " + str(accuracy*100) + '%')
